In [16]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [17]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split, Dataset

import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms

from tqdm import tqdm
print(torch.__version__)
print(torchvision.__version__)

In [18]:
input_size = 28
sequence_length = 28
num_classes = 10
num_layers = 2
hidden_size = 256
batch_size = 256
num_epochs = 3
lr = 1e-3
momentum = 0.9

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

In [64]:
class DigitDataset(Dataset):
    def __init__(self, df, labels, transform):
        self.data = df.astype(np.float32)
        self.labels = labels
        self.transform = transform
    
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img = np.reshape(self.data[idx], (28, 28), 'A')
        img = self.transform(img)
        if self.labels is not None:
            labels = self.labels[idx]
            return img, torch.from_numpy(np.asarray(labels)).type(torch.LongTensor)
        else:
            return img

In [55]:
class SequenceModels(nn.Module):
    def __init__(self, model, input_size, sequence_length, hidden_size, num_layers, num_classes):
        super(SequenceModels, self).__init__()
        self.types = model
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.model = self._create_model(model)
        self.fc = nn.Linear(
            sequence_length * hidden_size, num_classes
        )
    
    def forward(self, x):
        h0 = torch.rand(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.rand(self.num_layers, x.size(0), self.hidden_size).to(device)
        if self.types == "LSTM":
            out, _ = self.model(x, (h0, c0))
        else:
            out, _ = self.model(x, h0)

        out = out.reshape(x.shape[0], -1)
        out = self.fc(out)
        return out

    def _create_model(self, model):
        input_size = self.input_size
        hidden_size = self.hidden_size
        num_layers = self.num_layers
        if model == "RNN":
            model = nn.RNN(
                input_size, hidden_size, num_layers, batch_first=True)
        elif model == "GRU":
            model = nn.GRU(
                input_size, hidden_size, num_layers, batch_first=True)
        elif model == "LSTM":
            model = nn.LSTM(
                input_size, hidden_size, num_layers, batch_first=True)
        return model

In [56]:
model = SequenceModels(
    model = "LSTM",
    input_size = input_size,
    sequence_length = sequence_length,
    hidden_size = hidden_size,
    num_layers = num_layers,
    num_classes = num_classes).to(device)
print(model)

In [57]:
my_transforms = transforms.ToTensor()

train_path = "/kaggle/input/digit-recognizer/train.csv"
test_path = "/kaggle/input/digit-recognizer/test.csv"

main_df = pd.read_csv(train_path)
train_df = main_df.values[:, 1:]
labels = main_df.values[:, 0]

digit_dataset = DigitDataset(train_df, labels,  transform=my_transforms)
train_size = int(0.8 * len(digit_dataset))
val_size = len(digit_dataset) - train_size

train_set, val_set = random_split(digit_dataset, [train_size, val_size])
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False)

In [38]:
torch.from_numpy(np.asarray(labels)).type(torch.LongTensor)

In [58]:
def train(model, trainloader):
    loss_fn = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    for epoch in range(num_epochs):
        model.train()
        for X, y in tqdm(trainloader):
            X = X.to(device).squeeze(1)
            y = y.to(device)
            outputs = model(X)
            loss = loss_fn(outputs, y)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
    return model

In [59]:
def check_accuracy(model, testloader):
    model.eval()
    num_samples = len(testloader.dataset)
    correct = 0.0
    with torch.no_grad():
        for X, y in tqdm(testloader):
            X = X.to(device).squeeze(1)
            y = y.to(device)
            outputs = model(X)
            preds = outputs.max(1)[1]
            correct += torch.sum(preds == y).item()
    
    model.train()
    acc = correct / num_samples * 100
    return acc

In [60]:
trained_model = train(model, train_loader)
train_acc = check_accuracy(trained_model, train_loader)
val_acc = check_accuracy(trained_model, val_loader)
print(f"Accuracy in training set: {train_acc:.3f}")
print(f"Accuracy in validation set: {val_acc:.3f}")

In [70]:
test_df = pd.read_csv(test_path).values
test_set = DigitDataset(test_df, labels=None, transform=my_transforms)
test_loader = DataLoader(test_set, batch_size=batch_size)

In [77]:
test_preds = []

for inputs in test_loader:
    inputs = inputs.to(device).squeeze(1)
    outputs = model(inputs)
    _, preds = torch.max(outputs, 1)
    test_preds.append(preds.numpy())
    
predictions = []
for pred_batch in test_preds:
    for pred in pred_batch:
        predictions.append(pred)

In [81]:
submission = pd.DataFrame(
    {"ImageId": range(1, len(predictions) + 1),
     "Label": predictions})

submission.to_csv("submission.csv", index=False)